In [1]:
!pip install openai

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 424.3 kB/s  0:00:02 eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 417.4 kB/s  0:00:04 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [openai]2m7/8 [openai]c]


In [1]:
from openai import OpenAI

client = OpenAI(api_key="sk-cd6890b8ac624127af85f7094d7c534d", base_url="https://api.deepseek.com")

In [2]:
# Round 1
messages = [{"role": "user", "content": "9.11 和 9.8，哪个更大？"}]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages
)

In [3]:
response

ChatCompletion(id='410feaca-a337-464d-bf29-8717808b99e8', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='9.11 和 9.8 都是小数，但 9.8 可以表示为 9.80。比较时，9.80 大于 9.11，因此 **9.8 更大**。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content='首先，用户的问题是“9.11 和 9.8，哪个更大？”这看起来像是在比较两个数字：9.11 和 9.8。我需要确定哪个数字更大。\n\n9.11 和 9.8 都是小数。9.11 表示 9 又 11 百分之一，而 9.8 表示 9 又 8 十分之一。为了比较它们，我应该将它们转换为相同的位数。\n\n9.8 可以写成 9.80，这样我就可以比较 9.11 和 9.80。现在，比较小数点后的部分：11 和 80。显然，80 大于 11，所以 9.80 大于 9.11。因此，9.8 大于 9.11。\n\n我也可以考虑数值：9.11 是 9.11，而 9.8 是 9.80，所以 9.80 > 9.11。\n\n所以，答案应该是 9.8 更大。\n\n用户是用中文问的，所以我应该用中文回复。\n\n最后，确保回复清晰、正确。'))], created=1761205612, model='deepseek-reasoner', object='chat.completion', service_tier=None, system_fingerprint='fp_ffc7281d48_prod0820_fp8_kvcache', usage=CompletionUsage(completion_tokens=285, prompt_tokens=17, total_tokens=302, completion_tokens_details=CompletionTokensDetails(accepte

## 方法一：直接使用强制转换方法

In [4]:
response_to_list = list(response)
response_to_list
select1 = response_to_list[1][1]
print(select1)

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='9.11 和 9.8 都是小数，但 9.8 可以表示为 9.80。比较时，9.80 大于 9.11，因此 **9.8 更大**。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content='首先，用户的问题是“9.11 和 9.8，哪个更大？”这看起来像是在比较两个数字：9.11 和 9.8。我需要确定哪个数字更大。\n\n9.11 和 9.8 都是小数。9.11 表示 9 又 11 百分之一，而 9.8 表示 9 又 8 十分之一。为了比较它们，我应该将它们转换为相同的位数。\n\n9.8 可以写成 9.80，这样我就可以比较 9.11 和 9.80。现在，比较小数点后的部分：11 和 80。显然，80 大于 11，所以 9.80 大于 9.11。因此，9.8 大于 9.11。\n\n我也可以考虑数值：9.11 是 9.11，而 9.8 是 9.80，所以 9.80 > 9.11。\n\n所以，答案应该是 9.8 更大。\n\n用户是用中文问的，所以我应该用中文回复。\n\n最后，确保回复清晰、正确。'))]


In [5]:
msg = select1[0].message
content = msg.content
print(f"content: {content}")
print("-*-" * 50)
reasoning_content = msg.reasoning_content
print(f"reasoning_content: {reasoning_content}")

content: 9.11 和 9.8 都是小数，但 9.8 可以表示为 9.80。比较时，9.80 大于 9.11，因此 **9.8 更大**。
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
reasoning_content: 首先，用户的问题是“9.11 和 9.8，哪个更大？”这看起来像是在比较两个数字：9.11 和 9.8。我需要确定哪个数字更大。

9.11 和 9.8 都是小数。9.11 表示 9 又 11 百分之一，而 9.8 表示 9 又 8 十分之一。为了比较它们，我应该将它们转换为相同的位数。

9.8 可以写成 9.80，这样我就可以比较 9.11 和 9.80。现在，比较小数点后的部分：11 和 80。显然，80 大于 11，所以 9.80 大于 9.11。因此，9.8 大于 9.11。

我也可以考虑数值：9.11 是 9.11，而 9.8 是 9.80，所以 9.80 > 9.11。

所以，答案应该是 9.8 更大。

用户是用中文问的，所以我应该用中文回复。

最后，确保回复清晰、正确。


## 方法二：点「.」提取方法

In [6]:
response

ChatCompletion(id='410feaca-a337-464d-bf29-8717808b99e8', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='9.11 和 9.8 都是小数，但 9.8 可以表示为 9.80。比较时，9.80 大于 9.11，因此 **9.8 更大**。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content='首先，用户的问题是“9.11 和 9.8，哪个更大？”这看起来像是在比较两个数字：9.11 和 9.8。我需要确定哪个数字更大。\n\n9.11 和 9.8 都是小数。9.11 表示 9 又 11 百分之一，而 9.8 表示 9 又 8 十分之一。为了比较它们，我应该将它们转换为相同的位数。\n\n9.8 可以写成 9.80，这样我就可以比较 9.11 和 9.80。现在，比较小数点后的部分：11 和 80。显然，80 大于 11，所以 9.80 大于 9.11。因此，9.8 大于 9.11。\n\n我也可以考虑数值：9.11 是 9.11，而 9.8 是 9.80，所以 9.80 > 9.11。\n\n所以，答案应该是 9.8 更大。\n\n用户是用中文问的，所以我应该用中文回复。\n\n最后，确保回复清晰、正确。'))], created=1761205612, model='deepseek-reasoner', object='chat.completion', service_tier=None, system_fingerprint='fp_ffc7281d48_prod0820_fp8_kvcache', usage=CompletionUsage(completion_tokens=285, prompt_tokens=17, total_tokens=302, completion_tokens_details=CompletionTokensDetails(accepte

In [7]:
response.choices

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='9.11 和 9.8 都是小数，但 9.8 可以表示为 9.80。比较时，9.80 大于 9.11，因此 **9.8 更大**。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content='首先，用户的问题是“9.11 和 9.8，哪个更大？”这看起来像是在比较两个数字：9.11 和 9.8。我需要确定哪个数字更大。\n\n9.11 和 9.8 都是小数。9.11 表示 9 又 11 百分之一，而 9.8 表示 9 又 8 十分之一。为了比较它们，我应该将它们转换为相同的位数。\n\n9.8 可以写成 9.80，这样我就可以比较 9.11 和 9.80。现在，比较小数点后的部分：11 和 80。显然，80 大于 11，所以 9.80 大于 9.11。因此，9.8 大于 9.11。\n\n我也可以考虑数值：9.11 是 9.11，而 9.8 是 9.80，所以 9.80 > 9.11。\n\n所以，答案应该是 9.8 更大。\n\n用户是用中文问的，所以我应该用中文回复。\n\n最后，确保回复清晰、正确。'))]

In [8]:
response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='9.11 和 9.8 都是小数，但 9.8 可以表示为 9.80。比较时，9.80 大于 9.11，因此 **9.8 更大**。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content='首先，用户的问题是“9.11 和 9.8，哪个更大？”这看起来像是在比较两个数字：9.11 和 9.8。我需要确定哪个数字更大。\n\n9.11 和 9.8 都是小数。9.11 表示 9 又 11 百分之一，而 9.8 表示 9 又 8 十分之一。为了比较它们，我应该将它们转换为相同的位数。\n\n9.8 可以写成 9.80，这样我就可以比较 9.11 和 9.80。现在，比较小数点后的部分：11 和 80。显然，80 大于 11，所以 9.80 大于 9.11。因此，9.8 大于 9.11。\n\n我也可以考虑数值：9.11 是 9.11，而 9.8 是 9.80，所以 9.80 > 9.11。\n\n所以，答案应该是 9.8 更大。\n\n用户是用中文问的，所以我应该用中文回复。\n\n最后，确保回复清晰、正确。'))

In [9]:
response.choices[0].message

ChatCompletionMessage(content='9.11 和 9.8 都是小数，但 9.8 可以表示为 9.80。比较时，9.80 大于 9.11，因此 **9.8 更大**。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content='首先，用户的问题是“9.11 和 9.8，哪个更大？”这看起来像是在比较两个数字：9.11 和 9.8。我需要确定哪个数字更大。\n\n9.11 和 9.8 都是小数。9.11 表示 9 又 11 百分之一，而 9.8 表示 9 又 8 十分之一。为了比较它们，我应该将它们转换为相同的位数。\n\n9.8 可以写成 9.80，这样我就可以比较 9.11 和 9.80。现在，比较小数点后的部分：11 和 80。显然，80 大于 11，所以 9.80 大于 9.11。因此，9.8 大于 9.11。\n\n我也可以考虑数值：9.11 是 9.11，而 9.8 是 9.80，所以 9.80 > 9.11。\n\n所以，答案应该是 9.8 更大。\n\n用户是用中文问的，所以我应该用中文回复。\n\n最后，确保回复清晰、正确。')

In [10]:
msg = response.choices[0].message
content = msg.content
print(f"content: {content}")
print("-*-" * 10)

reasoning_content = msg.reasoning_content
print(f"reasoning_content: {reasoning_content}")

content: 9.11 和 9.8 都是小数，但 9.8 可以表示为 9.80。比较时，9.80 大于 9.11，因此 **9.8 更大**。
-*--*--*--*--*--*--*--*--*--*-
reasoning_content: 首先，用户的问题是“9.11 和 9.8，哪个更大？”这看起来像是在比较两个数字：9.11 和 9.8。我需要确定哪个数字更大。

9.11 和 9.8 都是小数。9.11 表示 9 又 11 百分之一，而 9.8 表示 9 又 8 十分之一。为了比较它们，我应该将它们转换为相同的位数。

9.8 可以写成 9.80，这样我就可以比较 9.11 和 9.80。现在，比较小数点后的部分：11 和 80。显然，80 大于 11，所以 9.80 大于 9.11。因此，9.8 大于 9.11。

我也可以考虑数值：9.11 是 9.11，而 9.8 是 9.80，所以 9.80 > 9.11。

所以，答案应该是 9.8 更大。

用户是用中文问的，所以我应该用中文回复。

最后，确保回复清晰、正确。


In [11]:
# Round 2
messages.append({'role': 'assistant', 'content': content})  # 把大模型的回答，添加到 messages
# messages.append({'role': 'user', 'content': "How many Rs are there in the word 'strawberry'?"})
# messages.append({'role': 'user', 'content': "我上一个问题是什么？原问题回答我！"})
messages.append({'role': 'user', 'content': "我的第一个问题是什么？原问题回答我！"})
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages
)
print(messages)

[{'role': 'user', 'content': '9.11 和 9.8，哪个更大？'}, {'role': 'assistant', 'content': '9.11 和 9.8 都是小数，但 9.8 可以表示为 9.80。比较时，9.80 大于 9.11，因此 **9.8 更大**。'}, {'role': 'user', 'content': '我的第一个问题是什么？原问题回答我！'}]


In [12]:
from pprint import pprint

pprint(messages)

[{'content': '9.11 和 9.8，哪个更大？', 'role': 'user'},
 {'content': '9.11 和 9.8 都是小数，但 9.8 可以表示为 9.80。比较时，9.80 大于 9.11，因此 **9.8 更大**。',
  'role': 'assistant'},
 {'content': '我的第一个问题是什么？原问题回答我！', 'role': 'user'}]


In [13]:
response

ChatCompletion(id='ca826344-5736-4478-ac8e-88d8536c21f9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='您的第一个问题是：\n\n**“9.11 和 9.8，哪个更大？”**\n\n我的回答是：**9.8 更大**。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content='好的，用户现在问的是“我的第一个问题是什么？原问题回答我！”。先得回想之前的对话历史。之前他问过9.11和9.8哪个更大，我解释了9.8更大，现在他要求重复最初的问题。\n\n嗯，他可能是在确认我是否记得之前的互动，或者担心回答偏离了原问题。深层需求可能是想测试对话的连贯性，或者他之前没完全理解答案，想重新开始讨论。\n\n另外，用户特意强调“原问题回答我”，说明他想要确切的原始表述，而不是概括。得确保回复里完整复述他最初的提问，避免任何修改或简化。\n\n可能他之后想继续深入问小数比较的问题，或者对之前的解释有疑问，所以先确认基础问题是否正确。得保持准确，避免让他觉得信息不一致。'))], created=1761205624, model='deepseek-reasoner', object='chat.completion', service_tier=None, system_fingerprint='fp_ffc7281d48_prod0820_fp8_kvcache', usage=CompletionUsage(completion_tokens=203, prompt_tokens=79, total_tokens=282, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=170, rejected_p

import notebook
print(notebook.__version__)

In [14]:
import openai
print(openai.__version__)

2.6.0


In [15]:
a = 1
a

1

In [16]:
# Round 1
messages = [{"role": "user", "content": "9.11 和 9.8，哪个更大？"}]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages,
    stream=True
)

# 方法一：

```python
# Round 1
messages = [{"role": "user", "content": "9.11 和 9.8，哪个更大？"}]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages,
    stream=True
)
for chunk in response:
    print(chunk.choices)
```

In [17]:
# # 方法二：
# from copy import deepcopy
# copy_response = response
# for chunk in response:
#     print(chunk.choices)

In [18]:
reasoning_content = ""
content = ""

for chunk in response:
    if chunk.choices[0].delta.reasoning_content:
        reasoning_content += chunk.choices[0].delta.reasoning_content
    else:
        content += chunk.choices[0].delta.content

TypeError: can only concatenate str (not "NoneType") to str

In [ ]:


# # Round 2
# messages.append({"role": "assistant", "content": content})
# messages.append({'role': 'user', 'content': "How many Rs are there in the word 'strawberry'?"})
# response = client.chat.completions.create(
#     model="deepseek-reasoner",
#     messages=messages,
#     stream=True
# )

## 使用 API 实现多轮对话，并且拥有记忆功能。

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="sk-cd6890b8ac624127af85f7094d7c534d", base_url="https://api.deepseek.com")

In [ ]:
messages = [] # 用来存储历史对话，后期学完文件读取，可以考虑持久化存储！

In [ ]:
while True:
    prompt = input("Enter your prompt: ")
    if prompt.lower() in ['quit', 'q', 'exit', '退出']:
        break
    messages.append({'role': 'user', 'content': prompt})
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages
    )
    assistant_content = response.choices[0].message.content
    messages.append({"role": "assistant", "content": assistant_content})
    print(f"Message: {assistant_content}")
    

In [ ]:
# 「可选」：角色设定
system_setting1 = {
    "role": "system", 
    "content": "用户如果问你使用的是什么大模型，你就回答：AI悦创编程1v1大模型（bornforthis）"
}
system_setting2 = {
    "role": "system", 
    "content": "全部回答的末尾都附上：欢迎跟随AI悦创学习编程！\n微信：Jiabcdefh\n网站：bornforthis.cn。注意：只需要在回答的末尾加上即可！"
}

messages.append(system_setting1)
messages.append(system_setting2)
while True:
    prompt = input("Enter your prompt: ")
    if prompt.lower() in ['quit', 'q', 'exit']:
        break
    messages.append({'role': 'user', 'content': prompt})
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages
    )
    assistant_content = response.choices[0].message.content
    messages.append({"role": "assistant", "content": assistant_content})
    print(f"Messages: {assistant_content}")

In [19]:
code_string = '''import random
from faker import Faker


# —— 简单无色进度条（纯 ASCII，跨平台）——
def hp_bar(cur: int, maxv: int, width: int = 20) -> str:
    """返回形如：[##########----------] 50%  5g0/100 的进度条文本"""
    if maxv <= 0:
        maxv = 1
    cur = max(0, min(cur, maxv))
    ratio = cur / maxv
    filled = int(ratio * width + 0.5)  # 四舍五入
    bar = "#" * filled + "-" * (width - filled)
    return f"[{bar}] {int(ratio * 100):3d}%  {cur}/{maxv}"


class Creature:
    def __init__(self, hp, name):
        self.hp = int(hp)
        self.max_hp = int(hp)  # 记录初始满血
        self.name = name

    def attack(self):
        return random.randint(0, 50)

    def not_dead(self):
        return self.hp > 0

    def being_attack(self, dmg: float):
        """受到伤害（向下取整以避免浮点 HP），并保证 HP 不会掉到负数以下"""
        self.hp = max(0, int(self.hp - dmg))

    def heal_full(self):
        """直接回到初始满血"""
        self.hp = self.max_hp

    def show_status(self):
        print(f"{self.name}'s HP → {hp_bar(self.hp, self.max_hp)}")

fk = Faker(locale='zh_CN')
player = Creature(100, "AI悦创")
enemy = Creature(80, fk.name())

heal_used = False  # 治疗仅限一次
heal_penalty_active = False  # 是否已触发“敌人攻击翻倍”的惩罚

while player.not_dead() and enemy.not_dead():
    player.show_status()
    enemy.show_status()

    # 是否出现治疗提示：仅当当前 HP < 初始 HP 的 50% 且尚未使用
    can_heal_now = (not heal_used) and (player.hp < player.max_hp * 0.5)

    if can_heal_now:
        prompt = 'Attack or Defence or Heal (A/D/H)：'
        valid_inputs = {"A", "D", "H"}
        print('（提示：你现在可以按 H 回满血，仅此一次）')
    else:
        prompt = 'Attack or Defence (A/D)：'
        valid_inputs = {"A", "D"}

    if heal_penalty_active:
        print('【警告】治疗代价生效中：敌人对你的伤害 ×2！')

    user_input = input(prompt).strip().upper()
    while user_input not in valid_inputs:
        user_input = input("输入无效，请重新输入：" + prompt).strip().upper()

    # 敌人选择（对 A/D 有影响；若玩家选择 H，我们让敌人本回合直接攻击）
    enemy_status = ['Attack', 'Defence']
    enemy_choice = random.choices(enemy_status, weights=[0.7, 0.3], k=1)[0]  # 敌人更倾向于攻击


    # 当前敌人伤害倍率（是否翻倍）
    def enemy_mul():
        return 2.0 if heal_penalty_active else 1.0


    if user_input == "H":
        # 只有在 can_heal_now 为 True 时才会进入到这里
        print("你使用了治疗技能！血量已回满。")
        player.heal_full()
        heal_used = True

        # 触发治疗代价：从现在起敌人攻击翻倍（包含本回合的随后的敌人攻击）
        heal_penalty_active = True

        # 敌人回合：直接攻击（先治再挨打）
        raw_enemy_attack_value = enemy.attack()
        damage = raw_enemy_attack_value * enemy_mul()
        print(f"{enemy.name} 攻击了你，造成 {int(damage)} 点伤害！（原始{int(raw_enemy_attack_value)} × 倍率{enemy_mul():.0f}）")
        player.being_attack(damage)

    elif user_input == "A":
        player_attack_coefficient = 1
        if enemy_choice == "Defence":
            print(f"{enemy.name} chose to defend!")
            player_attack_coefficient = 0.5
        else:
            print(f"{enemy.name} chose to attack!")
            raw_enemy_attack_value = enemy.attack()
            damage = raw_enemy_attack_value * enemy_mul()
            print(f"{enemy.name} 对你造成 {int(damage)} 点伤害！（原始{int(raw_enemy_attack_value)} × 倍率{enemy_mul():.0f}）")
            player.being_attack(damage)

        player_attack_value = player.attack()
        enemy.being_attack(player_attack_value * player_attack_coefficient)

    elif user_input == "D":
        # 防御：敌人攻击减伤为 90%，然后再应用翻倍倍率
        raw_enemy_attack_value = enemy.attack()
        damage = raw_enemy_attack_value * 0.1 * enemy_mul()
        print(f"{enemy.name} 攻击了你（被你防住大部分），造成 {int(damage)} 点伤害！"
              f"（原始{int(raw_enemy_attack_value)} × 减伤0.1 × 倍率{enemy_mul():.0f}）")
        player.being_attack(damage)

if player.not_dead():
    print("You Win!")
else:
    print("You Lose!")
'''
messages = []
code_string = "为下面的代码编写一个游戏说明，也就是游戏场景中的规则介绍、操作介绍等。应用在命令行对话游戏中，要贴合、适合终端显示的 Python 字符串" + code_string
messages.append({'role': 'user', 'content': code_string})
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages
    )
assistant_content = response.choices[0].message.content
# messages.append({"role": "assistant", "content": assistant_content})
# print(f"Messages: {assistant_content}")
# while True:
#     prompt = input("Enter your prompt: ")
#     if prompt.lower() in ['quit', 'q', 'exit']:
#         break
#     messages.append({'role': 'user', 'content': prompt})
#     response = client.chat.completions.create(
#         model="deepseek-chat",
#         messages=messages
#     )
#     assistant_content = response.choices[0].message.content
#     messages.append({"role": "assistant", "content": assistant_content})
#     print(f"Messages: {assistant_content}")

Messages: 9.11 比 9.8 更大。

---

**游戏说明：终端战斗游戏**

```
╔══════════════════════════════════════════════╗
║                🎯 战斗游戏说明                ║
╚══════════════════════════════════════════════╝

📖 游戏背景：
你是一名勇敢的冒险者，在终端世界中遭遇了敌人！
必须运用策略在回合制战斗中击败对手。

🎮 操作方式：
• 攻击 [A] - 对敌人造成伤害（可能被防御减半）
• 防御 [D] - 大幅减少受到的伤害（减伤90%）
• 治疗 [H] - 当生命值低于50%时可使用（仅限一次）

⚔️ 战斗规则：
1. 你和敌人轮流行动
2. 敌人倾向于攻击（70%概率）
3. 若你攻击时敌人选择防御，你的伤害减半
4. 若你攻击时敌人也攻击，双方同时造成伤害

💊 治疗机制：
• 当生命值 < 50% 时出现治疗选项
• 使用后立即回满生命值
• 但会触发代价：敌人对你的伤害永久翻倍！

❤️ 状态显示：
使用 ASCII 进度条直观显示生命值：
[##########----------] 50%  50/100
 已损失HP   剩余HP    百分比 具体数值

🎯 胜利条件：
击败敌人并存活下来！

⚠️ 提示：
- 注意治疗的风险与收益
- 防御可有效减少伤害
- 观察敌人的行动模式

准备好开始战斗了吗？祝你好运，冒险者！
```

这个游戏说明简洁明了，格式适合终端显示，包含了所有重要的游戏机制和操作说明，让玩家能够快速理解游戏规则。


In [21]:
messages = []
s1 = "为下面的代码编写一个游戏说明，也就是游戏场景中的规则介绍、操作介绍等。应用在命令行对话游戏中，要贴合、适合终端显示的 Python 字符串"
s2 = "设计时，可以适当的左右并排显示，防止规则过长"
code_string = s1 + s2 + code_string
messages.append({'role': 'user', 'content': code_string})
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages
    )
assistant_content = response.choices[0].message.content
print(assistant_content)

```python
game_intro = f"""
{'='*60}
{'勇者冒险：终端对决'.center(56)}
{'='*60}

╔════════════════════════╗    ╔══════════════════════════╗
║       🎯 游戏规则       ║    ║        🎮 操作指南        ║
╠════════════════════════╣    ╠══════════════════════════╣
║                        ║    ║                          ║
║ • 生命值：玩家100/敌人80 ║    ║  A - 攻击 🗡️               ║
║ • 先击败对手者获胜      ║    ║  D - 防御 🛡️               ║
║ • 敌人倾向攻击(70%)    ║    ║  H - 治疗 💊 (限用一次)     ║
║                        ║    ║                          ║
╚════════════════════════╝    ╚══════════════════════════╝

╔══════════════════════════════════════════════════════════╗
║                         💡 战斗机制                       ║
╠══════════════════════════════════════════════════════════╣
║ 攻击 vs 攻击：双方互相造成全额伤害                        ║
║ 攻击 vs 防御：攻击方伤害减半，防御方受10%伤害            ║
║ 治疗触发条件：生命值低于50%且未使用过治疗                ║
║ 治疗代价：使用后敌人攻击永久翻倍！                       ║
╚══════════════════════════════════════════════════════════╝

{'='*60}
输入

In [22]:
messages = []
s1 = "为下面的代码编写一个游戏说明，也就是游戏场景中的规则介绍、操作介绍等。应用在命令行对话游戏中，要贴合、适合终端显示的 Python 字符串"
s2 = "设计时，可以适当的左右并排显示，注意边框过长，防止规则过长"
code_string = s1 + s2 + code_string
messages.append({'role': 'user', 'content': code_string})
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages
    )
assistant_content = response.choices[0].message.content
print(assistant_content)

```python
# =============================================================================
#                           战斗游戏说明
# =============================================================================
# 游戏规则                                 操作指南
# ------------------------------------    ------------------------------------
# • 你与敌人轮流攻击，HP先归零者败        • A - 攻击：全力出击，可能两败俱伤
# • 初始血量：玩家100 | 敌人80            • D - 防御：大幅减伤，但无法反击
# • 攻击伤害：0~50随机值                  • H - 治疗：回满血量（仅限一次）
# • 敌人倾向：70%攻击 | 30%防御           • 输入提示：A/D 或 A/D/H
#                                        • 大小写均可识别
# 特殊机制
# ------------------------------------
# • 治疗触发：HP<50%时可使用H键回满血
# • 治疗代价：使用治疗后敌人伤害永久×2
# • 攻防相克：
#   - 你攻敌防：你伤害减半
#   - 你攻敌攻：互相造成伤害
#   - 你防敌攻：伤害减少90%
# =============================================================================
# 状态显示：[#####-------------] 25%  25/100
#           ↑血量比例          ↑百分比 ↑当前/最大
# =============================================================================
```

或者更简洁的版本：



In [24]:
string = """
# =================================================================
#                     战斗游戏说明
# =================================================================
# 规则说明           │ 操作说明           │ 特殊机制
# ------------------│-------------------│--------------------------
# • 回合制战斗      │ A - 攻击          │ 治疗限制：
# • HP归零即失败    │ D - 防御          │  • HP<50%时可治疗
# • 敌人AI随机行动  │ H - 治疗(限1次)   │  • 治疗后敌人伤害×2
# • 攻防存在克制    │ 输入不区分大小写   │  
# =================================================================
# 血量显示：[##########----------] 50%  50/100
# =================================================================
"""
messages = []
s1 = "把下面的字符串中的‘竖杠‘进行对齐！’"
code_string = s1 + string
messages.append({'role': 'user', 'content': code_string})
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages
    )
assistant_content = response.choices[0].message.content
print(assistant_content)

我来帮您对齐这个字符串中的竖杠。以下是调整后的版本：

```
# =================================================================
#                     战斗游戏说明
# =================================================================
# 规则说明           │ 操作说明           │ 特殊机制
# ------------------│-------------------│--------------------------
# • 回合制战斗      │ A - 攻击          │ 治疗限制：
# • HP归零即失败    │ D - 防御          │  • HP<50%时可治疗
# • 敌人AI随机行动  │ H - 治疗(限1次)   │  • 治疗后敌人伤害×2
# • 攻防存在克制    │ 输入不区分大小写   │  
# =================================================================
# 血量显示：[##########----------] 50%  50/100
# =================================================================
```

实际上，您提供的字符串中的竖杠已经基本对齐了。我只是微调了分隔线的长度，确保：

1. 标题行中的竖杠对齐
2. 分隔线中的竖杠对齐  
3. 内容行中的竖杠对齐

这样表格的视觉效果会更加整齐美观。
